<a href="https://colab.research.google.com/github/ak4721269/spam-email-classifier/blob/main/Spam_email_classifier_using_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the required packages

In [ ]:
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Upload the zip file of dataset and unzip it by running it below cell.

In [ ]:
import tarfile
import os

# Specify the path to your .tar.gz file
tar_file_path = '/content/enron1.tar.gz'

# Specify the path where you want to extract the files
extract_path = '/content/dataset'

# Create the extract folder if it doesn't exist
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Open the .tar.gz file
with tarfile.open(tar_file_path, 'r:gz') as tar:
    # Extract all contents to the specified folder
    tar.extractall(path=extract_path)

print(f"Files extracted to {extract_path}")


Files extracted to /content/dataset


In [ ]:
def load_data():
    print("Loading data...")

    ham_files_location = os.listdir("/content/dataset/enron1/ham")
    spam_files_location = os.listdir("/content/dataset/euron1/spam")
    data = []

In [ ]:
import os

def load_data():
    print("Loading data...")

    data = []

    # Load ham email
    ham_path = "/content/dataset/enron1/ham"
    for file_path in os.listdir(ham_path):
        file_path = os.path.join(ham_path, file_path)
        with open(file_path, "r", encoding="latin-1") as f:
            text = f.read()
            data.append([text, "ham"])

    # Load spam email
    spam_path = "/content/dataset/enron1/spam"
    for file_path in os.listdir(spam_path):
        file_path = os.path.join(spam_path, file_path)
        with open(file_path, "r", encoding="latin-1") as f:
            text = f.read()
            data.append([text, "spam"])

    data = np.array(data)
    print("flag 1: loaded data")
    return data


In [ ]:
def preprocess_data(data):
    print("Preprocessing data...")

    punc = string.punctuation           # Punctuation list
    sw = stopwords.words('english')
    for record in data:
        # Remove common punctuation and symbols
       for item in punc:
           record[0] = record[0].replace(item, "")
           # Lowercase all letters and remove stopwords
           splittedWords = record[0].split()
           newText = ""
           for word in splittedWords:
              if word not in sw:
                  word = word.lower()
                  newText = newText + " " + word
           record[0] = newText

    print("flag 2: preprocessed data")
    return data


In [ ]:
# Splitting original dataset into training dataset and test dataset
def split_data(data):
    print("Splitting data...")

    features = data[:, 0]   # array containing all email text bodies
    labels = data[:, 1]     # array containing corresponding labels
    print(labels)
    training_data, test_data, training_labels, test_labels =\
        train_test_split(features, labels, test_size = 0.27, random_state = 42)

    print("flag 3: splitted data")
    return training_data, test_data, training_labels, test_labels

In [ ]:
def get_count(text):
    wordCounts = dict()
    for word in text.split():
        if word in wordCounts:
            wordCounts[word] += 1
        else:
            wordCounts[word] = 1

    return wordCounts

In [ ]:
def euclidean_difference(test_WordCounts, training_WordCounts):
    total = 0
    for word in test_WordCounts:
        if word in test_WordCounts and word in training_WordCounts:
            total += (test_WordCounts[word] - training_WordCounts[word])**2
            del training_WordCounts[word]
        else:
            total += test_WordCounts[word]**2
    for word in training_WordCounts:
            total += training_WordCounts[word]**2

    return total**0.5


In [ ]:
def get_class(selected_Kvalues):
    spam_count = 0
    ham_count = 0
    for value in selected_Kvalues:
      if value[0] == "spam":
          spam_count += 1
      else:
          ham_count += 1

    if spam_count > ham_count:
        return "spam"
    else:
        return "ham"

In [ ]:
def knn_classifier(training_data, training_labels, test_data, K, tsize):
    print("Running KNN Classifier...")

    result = []
    counter = 1
    # word counts for training email
    training_WordCounts = []
    for training_text in training_data:
            training_WordCounts.append(get_count(training_text))
    for test_text in test_data:
        similarity = [] # List of euclidean distances
        test_WordCounts = get_count(test_text)  # word counts for test email

        # Getting euclidean difference
        for index in range(len(training_data)):
              euclidean_diff =\
                    euclidean_difference(test_WordCounts, training_WordCounts[index])
              similarity.append([training_labels[index], euclidean_diff])
    # Sort list in ascending order based on euclidean difference
        similarity = sorted(similarity, key = lambda i:i[1])

         # Select K nearest neighbours
        selected_Kvalues = []
        for i in range(K):
            selected_Kvalues.append(similarity[i])

    # Predicting the class of email
        result.append(get_class(selected_Kvalues))
    return result





In [ ]:
def main(K):
    data = load_data()
    data = preprocess_data(data)
    training_data, test_data, training_labels, test_labels = split_data(data)
    tsize = len(test_data)
    result = knn_classifier(training_data, training_labels, test_data[:tsize], K, tsize)
    accuracy = accuracy_score(test_labels[:tsize], result)
    print("training data size\t: " + str(len(training_data)))
    print("test data size\t\t: " + str(len(test_data)))
    print("K value\t\t\t\t: " + str(K))
    print("Samples tested\t\t: " + str(tsize))
    print("% accuracy\t\t\t: " + str(accuracy * 100))
    print("Number correct\t\t: " + str(int(accuracy * tsize)))
    print("Number wrong\t\t: " + str(int((1 - accuracy) * tsize)))

In [ ]:
main(11)

Loading data...
flag 1: loaded data
Preprocessing data...
flag 2: preprocessed data
Splitting data...
['ham' 'ham' 'ham' ... 'spam' 'spam' 'spam']
flag 3: splitted data
Running KNN Classifier...
training data size	: 3775
test data size		: 1397
K value				: 11
Samples tested		: 1397
% accuracy			: 72.15461703650679
Number correct		: 1008
Number wrong		: 389
